In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt
import time
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import torch
import torch.nn as nn
import random
import torch.nn.init
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from sklearn.model_selection import StratifiedShuffleSplit
from torch.optim import lr_scheduler
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from multiprocessing import cpu_count
from torch.utils.data import random_split
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# device setting
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' device:', device)

In [ ]:
# Define the custom dataset
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        filename = self.img_labels.iloc[idx, 0]
        image = Image.open(os.path.join(self.img_dir, filename)).convert('RGB')
        label = self.img_labels.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
        return image, label

    def get_labels(self):
        return self.img_labels.iloc[:, 1].tolist()

In [ ]:
# CustomSubset 클래스 정의
class CustomSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        image, label = self.subset[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((256,256)),
        transforms.RandomRotation(45),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
test_transform = transforms.Compose([
     transforms.Resize((256,256)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
# train
annotation_file = '/content/drive/MyDrive/uou-ie-g-03785-spring-2024-term-project/train.csv'
img_dir = '/content/drive/MyDrive/uou-ie-g-03785-spring-2024-term-project/Images/Train'
train_custom = CustomImageDataset(annotation_file, img_dir)
# train_custom = CustomImageDataset(annotation_file, img_dir, transform=train_transform)

In [ ]:
# 전체 데이터셋의 길이
dataset_size = len(train_custom)

# 8:2 비율로 데이터셋을 분할
train_size = int(dataset_size * 0.8)
valid_size = dataset_size - train_size

# 데이터셋을 랜덤하게 분할
train_data, valid_data = random_split(train_custom, [train_size, valid_size])

# 데이터셋 크기 확인
print(f"Train dataset size: {len(train_data)}")
print(f"Validation dataset size: {len(valid_data)}")

In [ ]:
# 각각의 데이터셋에 맞는 변환 적용
train_dataset = CustomSubset(train_data, transform=train_transform)
valid_dataset = CustomSubset(valid_data, transform=test_transform)

In [ ]:
# test
annotation_file2 = '/content/drive/MyDrive/uou-ie-g-03785-spring-2024-term-project/sample_submission.csv'
img_dir2 = '/content/drive/MyDrive/uou-ie-g-03785-spring-2024-term-project/Images/Test'
test_custom = CustomImageDataset(annotation_file2, img_dir2, transform=test_transform)

In [ ]:
num_workers = int(cpu_count() / 2)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, num_workers=num_workers, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, num_workers=num_workers, shuffle=False)
test_loader = DataLoader(test_custom, batch_size=32, shuffle=False)

In [ ]:
model = models.wide_resnet101_2(pretrained=True)
num_ftrs = model.fc.in_features
half_in_size = round(num_ftrs/2)
layer_width = 1024
Num_class=4

class SpinalNet(nn.Module):
    def __init__(self, Input_Size, Number_of_Split, HL_width, number_HL, Output_Size, Activation_Function):

        super(SpinalNet, self).__init__()
        Splitted_Input_Size = int(np.round(Input_Size/Number_of_Split))
        self.lru = Activation_Function
        self.fc1 = nn.Linear(Splitted_Input_Size, HL_width)
        if number_HL>1:
            self.fc2 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>2:
            self.fc3 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>3:
            self.fc4 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>4:
            self.fc5 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>5:
            self.fc6 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>6:
            self.fc7 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>7:
            self.fc8 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>8:
            self.fc9 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>9:
            self.fc10 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>10:
            self.fc11 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>11:
            self.fc12 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>12:
            self.fc13 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>13:
            self.fc14 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>14:
            self.fc15 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>15:
            self.fc16 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>16:
            self.fc17 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>17:
            self.fc18 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>18:
            self.fc19 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>19:
            self.fc20 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>20:
            self.fc21 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>21:
            self.fc22 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>22:
            self.fc23 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>23:
            self.fc24 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>24:
            self.fc25 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>25:
            self.fc26 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>26:
            self.fc27 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>27:
            self.fc28 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>28:
            self.fc29 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)
        if number_HL>29:
            self.fc30 = nn.Linear(Splitted_Input_Size+HL_width, HL_width)

        self.fcx = nn.Linear(HL_width*number_HL, Output_Size)

    def forward(self, x):
        x_all =x

        Splitted_Input_Size = self.fc1.in_features
        HL_width = self.fc2.in_features - self.fc1.in_features
        number_HL = int(np.round(self.fcx.in_features/HL_width))
        length_x_all = number_HL*Splitted_Input_Size

        while x_all.size(dim=1) < length_x_all:
            x_all = torch.cat([x_all, x],dim=1)

        x = self.lru(self.fc1(x_all[:,0:Splitted_Input_Size]))
        x_out = x

        counter1 = 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc2(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc3(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc4(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc5(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc6(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc7(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc8(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc9(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc10(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc11(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc12(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc13(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc14(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc15(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc16(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc17(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc18(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc19(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc20(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)
        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc21(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc22(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc23(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc24(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc25(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc26(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc27(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc28(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc29(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)

        counter1 = counter1 + 1
        if number_HL>counter1:
            x_from_all = x_all[:,Splitted_Input_Size* counter1:Splitted_Input_Size*(counter1+1)]
            x = self.lru(self.fc30(torch.cat([x_from_all, x], dim=1)))
            x_out = torch.cat([x_out, x], dim=1)
        #print("Size before output layer:",x_out.size(dim=1))
        x = self.fcx(x_out)
        return x

In [ ]:
model.fc = SpinalNet(Input_Size = num_ftrs, Number_of_Split =2, HL_width=1024, number_HL=30, Output_Size=Num_class, Activation_Function = nn.ReLU(inplace=True))

In [ ]:
model.to(device)

In [ ]:
class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

In [ ]:
class_weights = [1.0, 1.0, 1.2, 1.3]
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
class_weights

In [ ]:
epochs = 50
lr = 0.0001
criterion = nn.CrossEntropyLoss(weight=class_weights)
base_optimizer = torch.optim.Adam
optimizer = SAM(model.parameters(), base_optimizer, lr=lr)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1)

In [ ]:
# Early stopping parameters
early_stopping_patience = 7
best_valid_loss = float('inf')
patience_counter = 0

for epoch in range(epochs):
    model.train()

    train_loss = 0
    train_correct = 0
    tqdm_dataset = tqdm(train_loader)
    for x, y in tqdm_dataset:
        x = x.to(device)
        y = y.to(device)

        # Define the closure function
        def closure():
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            return loss, outputs

        # Perform the first step
        loss, outputs = closure()
        optimizer.first_step(zero_grad=True)

        # Perform the second step
        _, outputs = closure()
        optimizer.second_step(zero_grad=True)

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_correct += predicted.eq(y).sum().item()

        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Loss': '{:06f}'.format(loss.item()),
        })

    train_loss = train_loss / len(train_loader)
    train_acc = train_correct / len(train_loader.sampler)

    model.eval()

    valid_loss = 0
    valid_correct = 0
    tqdm_dataset = tqdm(valid_loader)
    with torch.no_grad():
        for x, y in tqdm_dataset:
            x = x.to(device)
            y = y.to(device)

            outputs = model(x)
            loss = criterion(outputs, y)
            valid_loss += loss.item()
            _, predicted = outputs.max(1)
            valid_correct += predicted.eq(y).sum().item()

            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Loss': '{:06f}'.format(loss.item()),
            })

    valid_loss = valid_loss / len(valid_loader)
    valid_acc = valid_correct / len(valid_loader.sampler)

    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.6f}, Train Acc: {train_acc:.6f}, Valid Loss: {valid_loss:.6f}, Valid Acc: {valid_acc:.6f}')

    # Move the scheduler step after optimizer step
    exp_lr_scheduler.step(valid_loss)

    # Check early stopping condition
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        patience_counter = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print(f'Early stopping triggered after {epoch+1} epochs.')
            break

In [ ]:
path = 'best_model.pth'
model.load_state_dict(torch.load(path))

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/uou-ie-g-03785-spring-2024-term-project/sample_submission.csv')

model.eval()

batch_index = 0

for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    outputs = model(images)
    batch_index = i * 32
    max_vals, max_indices = torch.max(outputs, 1)
    sample_submission.iloc[batch_index:batch_index + 32, 1:] = max_indices.long().cpu().numpy()[:,np.newaxis]

In [ ]:
sample_submission.to_csv('submit.csv',index=False)

In [ ]:
from google.colab import files

# submission.csv 파일 다운로드
files.download('submit.csv')